# Agent CFO — Performance Optimization & Design

---
This is Group 8's notebook for the ICT3113 Performance Testing and Optimization project following the required structure.


You will design and optimize an Agent CFO assistant for a listed company. The assistant should answer finance/operations questions using RAG (Retrieval-Augmented Generation) + agentic reasoning, with response time (latency) as the primary metric.

Your system must:
*   Ingest the company’s public filings.
*   Retrieve relevant passages efficiently.
*   Compute ratios/trends via tool calls (calculator, table parsing).
*   Produce answers with valid citations to the correct page/table.


## 1. Config & Secrets

Fill in your API keys in secrets. **Do not hardcode keys** in cells.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
COMPANY_NAME = "NVIDIA"

## 2. Data Download (Dropbox)

*   Annual Reports: last 3–5 years.
*   Quarterly Results Packs & MD&A (Management Discussion & Analysis).
*   Investor Presentations and Press Releases.
*   These files must be submitted later as a deliverable in the Dropbox data pack.
*   Upload them under `/content/data/`.

Scope limit: each team will ingest minimally 15 PDF files total.


## 3. System Requirements

**Retrieval & RAG**
*   Use a vector index (e.g., FAISS, LlamaIndex) + a keyword filter (BM25/ElasticSearch).
*   Citations must include: report name, year, page number, section/table.

**Agentic Reasoning**
*   Support at least 3 tool types: calculator, table extraction, multi-document compare.
*   Reasoning must follow a plan-then-act pattern (not a single unstructured call).

**Instrumentation**
*   Log timings for: T_ingest, T_retrieve, T_rerank, T_reason, T_generate, T_total.
*   Log: tokens used, cache hits, tools invoked.
*   Record p50/p95 latencies.

In [2]:
# Install dependencies for ingestion pipeline
%pip install langchain-community langchain-openai faiss-cpu rank-bm25 openai tiktoken pypdf pdfplumber

     ---------------------------------------- 0.0/42.8 kB ? eta -:--:--
     ---------------------------------------- 42.8/42.8 kB 1.0 MB/s eta 0:00:00
     ---------------------------------------- 0.0/48.5 kB ? eta -:--:--
     ---------------------------------------- 48.5/48.5 kB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/43.0 kB ? eta -:--:--
     ---------------------------------------- 43.0/43.0 kB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   --------------------- ------------------ 1.4/2.5 MB 29.4 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 32.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/76.0 kB ? eta -:--:--
   ---------------------------------------- 76.0/76.0 kB 4.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   ---- ----------------------------------- 2.1/18.2 MB 43.8 MB/s eta 0:00:01
   ----------- ----------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.17.0 requires numpy<2.0.0,>=1.26.0; python_version >= "3.12", but you have numpy 2.2.6 which is incompatible.
tensorflow-intel 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.4 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install pyinstrument

   ---------------------------------------- 0.0/125.0 kB ? eta -:--:--
   ---------------------------------------  122.9/125.0 kB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 125.0/125.0 kB 2.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Import libraries for ingestion pipeline

In [4]:
import time
import glob
import json
import re
import pdfplumber
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain.schema import Document
from langchain.retrievers import SVMRetriever
from langchain_core.retrievers import BaseRetriever
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.tools import StructuredTool
from langchain.agents import initialize_agent, AgentType
from typing import List, Dict, Optional

PDF Parser and Chunking (Using pdfplumber)
1. Extracts words from pdf
2. Detects sections
3. Returns langchain Document object with page content and metadata containing page number, report name, section, and type of content

In [5]:
def detect_section(page, text, min_size_diff=2.0):
    """
    Try to extract the section title from page text using regex patterns or font size, weight.
    Handles 10-K, 10Q, Presentations, Investor PDFs and similar formats.
    """
    patterns = [
        r"(Item\s+\d+[A-Za-z]?.\s*[A-Z][^\n]+)",  # e.g. "Item 7. Management’s Discussion..."
        r"(ITEM\s+\d+[A-Z]?.\s*[A-Z][^\n]+)",     # uppercase variant
        r"(^[A-Z][A-Z\s]{10,})"                   # fallback for all-caps section titles
    ]
    for p in patterns:
        m = re.search(p, text)
        if m:
            return m.group(1).strip()

    # For presentation decks/investor pdfs
    try:
        words = page.extract_words(extra_attrs=["size", "fontname"])
        if not words:
            return None

        from collections import defaultdict
        lines = defaultdict(list)

        # Group words by y-position
        for w in words:
            lines[round(w["top"], -1)].append(w)

        # Compute global average font size to compare against
        avg_font_size = sum(float(w["size"]) for w in words) / len(words)

        # Define line scoring function
        def score_line(line):
            avg_size = sum(float(w["size"]) for w in line) / len(line)
            bold_bonus = any("Bold" in w["fontname"] or "Black" in w["fontname"] for w in line)
            return avg_size + (3 if bold_bonus else 0), avg_size

        # Score all lines
        scored_lines = [(y, line, *score_line(line)) for y, line in lines.items()]
        if not scored_lines:
            return None

        # Pick the top scoring line
        _, best_line, best_score, best_size = max(scored_lines, key=lambda x: x[2])

        # If the line isn't significantly larger or bold, discard it
        if best_size < avg_font_size + min_size_diff:
            # (e.g. all text is 10pt, best line is 11pt → not a section)
            return None

        # Skip lines that look too long (to avoid full sentences)
        if len(best_line) > 20:
            return None

        section_title = " ".join(w["text"] for w in best_line)
        return section_title.strip() if section_title else None

    except Exception:
        return None

    return None


def parse_pdf_with_tables(pdf_path, report_name, year=None):
    text_docs, table_docs = [], []
    current_section = None  # rolling context

    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            text = page.extract_text() or ""
            tables = page.extract_tables() or []

            # --- Try to detect section header ---
            section = detect_section(page, text)
            if section:
                current_section = section  # update rolling section

            # --- Create text Document ---
            text_docs.append(
                Document(
                    page_content=text,
                    metadata={
                        "page": i,
                        "report": report_name,
                        "year": year,
                        "section": current_section,
                        "type": "text"
                    }
                )
            )

            # --- Create table Documents ---
            for t in tables:
                cleaned_table = [
                    [cell if cell is not None else "" for cell in row]
                    for row in t
                ]
                table_text = "\n".join(["\t".join(row) for row in cleaned_table])
                table_docs.append(
                    Document(
                        page_content=table_text,
                        metadata={
                            "page": i,
                            "report": report_name,
                            "year": year,
                            "section": current_section,
                            "type": "table"
                        }
                    )
                )

    return text_docs, table_docs

Main ingestion pipeline

1.   Loops over all pdfs in folder
2.   Checks cache for parsed pdf
3.   Load from cache if exists, avoiding re-parsing pdf
4.   Save to cache after parsing
5.   Use parallel processing to parse pdfs in parallel



In [6]:
# --- Parameters ---
pdf_folder = "data/"

PARSED_DIR = "cache/parsed_pdfs"
os.makedirs(PARSED_DIR, exist_ok=True)

def parse_pdf_cached(pdf_path, report_name, year=None):
    cache_file = os.path.join(PARSED_DIR, f"{report_name}.json")

    try:
        # --- If cached and file not modified, skip re-parsing ---
        pdf_mtime = os.path.getmtime(pdf_path)
        if os.path.exists(cache_file):
            # cache_mtime = os.path.getmtime(cache_file)
            # if cache_mtime > pdf_mtime:
            print(f"Using cached parse for {report_name}\n")
            with open(cache_file, "r", encoding="utf-8") as f:
                data = json.load(f)
            docs = [Document(**d) for d in data]
            return docs

        # --- Otherwise, parse fresh ---
        print(f"Parsing {os.path.basename(pdf_path)} ...\n")
        text_docs, table_docs = parse_pdf_with_tables(pdf_path, report_name, year)
        docs = text_docs + table_docs

        # Save to cache
        serializable_docs = [
            {"page_content": d.page_content, "metadata": d.metadata}
            for d in docs
        ]
        with open(cache_file, "w", encoding="utf-8") as f:
            json.dump(serializable_docs, f, ensure_ascii=False, indent=2)

        return docs
    except Exception as e:
        print(f"Error parsing PDF {report_name}: {e}")
        return []


# --- Measure time!! ----
start_total = time.perf_counter()
all_docs = []

def process_pdf(pdf_path):
    report_name = os.path.basename(pdf_path).replace(".pdf", "")
    return parse_pdf_cached(pdf_path, report_name)

for pdf_path in glob.glob(os.path.join(pdf_folder, "*.pdf")):
    doc = process_pdf(pdf_path)
    all_docs.extend(doc)

# with ThreadPoolExecutor(max_workers=4) as executor:
#     results = list(executor.map(process_pdf, glob.glob(os.path.join(pdf_folder, "*.pdf"))))
#     for docs in results:
#         all_docs.extend(docs)

elapsed_total = time.perf_counter() - start_total
print(f"Loaded {len(all_docs)} parsed documents in {elapsed_total:.2f}s.")


Parsing FY21_10K.pdf ...

Parsing FY22_10K.pdf ...

Parsing FY23_10K.pdf ...

Parsing FY24Q1_10Q.pdf ...

Parsing FY24Q2_10Q.pdf ...

Parsing FY24Q3_10Q.pdf ...



Cannot set gray non-stroke color because /'Pattern1' is an invalid float value
Cannot set gray non-stroke color because /'Pattern2' is an invalid float value
Cannot set gray non-stroke color because /'Pattern3' is an invalid float value
Cannot set gray non-stroke color because /'Pattern4' is an invalid float value
Cannot set gray non-stroke color because /'Pattern5' is an invalid float value
Cannot set gray non-stroke color because /'Pattern6' is an invalid float value
Cannot set gray non-stroke color because /'Pattern7' is an invalid float value
Cannot set gray non-stroke color because /'Pattern8' is an invalid float value
Cannot set gray non-stroke color because /'Pattern9' is an invalid float value
Cannot set gray non-stroke color because /'Pattern10' is an invalid float value
Cannot set gray non-stroke color because /'Pattern11' is an invalid float value
Cannot set gray non-stroke color because /'Pattern12' is an invalid float value
Cannot set gray non-stroke color because /'Patter

Parsing FY24_10K.pdf ...

Parsing FY25Q1_10Q.pdf ...

Parsing FY25Q2_10Q.pdf ...



Cannot set gray non-stroke color because /'P25' is an invalid float value
Cannot set gray non-stroke color because /'P46' is an invalid float value


Parsing FY25Q2_QuarterlyPresentation.pdf ...



...

Parsing FY26Q2_PR.pdf ...



Cannot set gray non-stroke color because /'P25' is an invalid float value
Cannot set gray non-stroke color because /'P46' is an invalid float value


Parsing FY26Q2_QuarterlyPresentation.pdf ...



Cannot set gray non-stroke color because /'P129' is an invalid float value
Cannot set gray non-stroke color because /'P135' is an invalid float value
Cannot set gray non-stroke color because /'P143' is an invalid float value
Cannot set gray non-stroke color because /'P227' is an invalid float value


Loaded 2039 parsed documents in 658.26s.


Chunking
1. Uses langchain's Recursive Character Text splitter to split documents
2. Returns chunked documents

In [7]:
from pyinstrument import Profiler
prof = Profiler(async_mode="disabled")
prof.start()

start_total = time.perf_counter()

max_chunk_size = 800
chunk_overlap = 100

def chunk_documents(all_docs, chunk_size=max_chunk_size, chunk_overlap=chunk_overlap):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return splitter.split_documents(all_docs)

chunked_docs = chunk_documents(all_docs, max_chunk_size, chunk_overlap)
print(f"Chunked into {len(chunked_docs)} total segments.")

print(f"Parsed and chunked {len(all_docs)} total text segments from {len(glob.glob(pdf_folder + '*.pdf'))} PDFs.")
elapsed_total = time.perf_counter() - start_total

prof.stop()
print(prof.output_text(unicode=True, color=True))
prof.last_session.save("01_Chunking.pyisession")
html_output = prof.output_html()
with open("01_Chunking.html", "w", encoding="utf-8") as f:
    f.write(html_output)
print(f"Total elapsed time: {elapsed_total}")

Chunked into 6354 total segments.
Parsed and chunked 2039 total text segments from 21 PDFs.

  _     ._   __/__   _ _  _  _ _/_   Recorded: 02:10:49  Samples:  398
 /_//_/// /_\ / //_// / //_'/ //     Duration: 0.717     CPU time: 0.672
/   _/                      v5.1.1

Profile at C:\Users\cryst\AppData\Local\Temp\ipykernel_12512\492970387.py:3

0.716 ZMQInteractiveShell.run_ast_nodes  IPython\core\interactiveshell.py:3418
└─ 0.715 <module>  ..\..\..\Temp\ipykernel_12512\492970387.py:1
   └─ 0.714 chunk_documents  ..\..\..\Temp\ipykernel_12512\492970387.py:10
      └─ 0.713 RecursiveCharacterTextSplitter.split_documents  langchain_text_splitters\base.py:97
            [27 frames hidden]  langchain_text_splitters, langchain_c...
               0.346 Document.__init__  langchain_core\load\serializable.py:113
               └─ 0.337 [self]  langchain_core\load\serializable.py


Total elapsed time: 0.7155587999950512


## 4. Baseline Pipeline

**Baseline (starting point)**
*   Naive chunking.
*   Single-pass vector search.
*   One LLM call, no caching.

In [ ]:
# # TODO: Implement baseline retrieval + generation
# # Take a query -> Retrieve relevant chunks -> Feed to LLM -> answer

# # number of docs to retrieve per query
# bm25_k = 10
# faiss_k = 12

# # FAISS Vector Index
# faiss_store = FAISS.from_documents(chunked_docs, OpenAIEmbeddings(model="text-embedding-3-small"))
# faiss_store.save_local("faiss_index")

# print("FAISS vector index built and saved locally.")

# # BM25 Keyword Index
# bm25_retriever = BM25Retriever.from_documents(chunked_docs)
# bm25_retriever.k = bm25_k

# print("BM25 keyword retriever ready.")

# print("Ingestion complete — FAISS and BM25 indices are ready.\n")

# # Test Prompt
# question = "Show Operating Expenses for the last 3 fiscal years, year-on-year comparison."

# # Test FAISS vector index
# faiss_retriever = faiss_store.as_retriever(search_kwargs={"k": faiss_k})
# test_faiss_docs = faiss_retriever.invoke(question)
# print('Using FAISS')
# print(test_faiss_docs[0].page_content)
# print(test_faiss_docs[0].metadata)
# print('\n')

# # Test BM25 retriever
# test_bm25_docs = bm25_retriever.invoke(question)
# print('Using BM25')
# print(test_bm25_docs[0].page_content)
# print(test_bm25_docs[0].metadata)
# print('\n')

# # Test SVM retriever
# # svm_retriever = SVMRetriever.from_documents(chunked_docs, embeddings=embedding_model)
# # svm_retriever.k = 10
# # test_svm_docs = svm_retriever.invoke(question)
# # print('Using SVM:')
# # print(test_svm_docs[0].page_content)
# # print(test_svm_docs[0].metadata)

# context = "\n\n".join([doc.page_content for doc in test_faiss_docs])

In [ ]:
# for doc in test_faiss_docs:
#   print("============================")
#   print(doc.page_content)
#   print(doc.metadata)

In [ ]:
# # Set Prompts
# from langchain_core.prompts import PromptTemplate

# prompt_template = \
# """
# You are a helpful assistant for question answering tasks.
# Use the following pieces of retrieved context to answer the given question.
# If you dont know the answer, just say that you dont know.
# Use up to three sentenses to keep answer precise.

# Question: {question}

# Context: {context}
# """

# prompt_template = PromptTemplate.from_template(prompt_template)
# # Invoke and pass in query and context
# prompt = prompt_template.invoke(
#     {
#         "context": context,
#         "question": question
#     }
# )
# prompt.text

In [ ]:
# # Generate Response
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# response = llm.invoke(prompt)

# # Parse the response into well formated text
# from langchain_core.output_parsers import StrOutputParser
# parser = StrOutputParser()

# res = parser.invoke(response)

# print(res)

## 4b. Agentic RAG

In [15]:
# Retriever Tool
class RetrieverTool:
    """Wrapper around FAISS retriever for LangChain/OpenAI tools interface."""

    def __init__(self, faiss_store: FAISS, top_k: int = 12):
        self.retriever: BaseRetriever = faiss_store.as_retriever(
            search_kwargs={"k": top_k}
        )

    def forward(self, query: str) -> str:
        """Retrieve top-k documents similar to the query."""
        assert isinstance(query, str), "Your search query must be a string"
        docs = self.retriever.invoke(query)

        if not docs:
            return "No relevant documents found."

        formatted_docs = "\n".join(
            [
                f"===== Document {i+1} =====\n"
                f"{doc.page_content}\n"
                f"Metadata: {doc.metadata}\n"
                for i, doc in enumerate(docs)
            ]
        )
        return formatted_docs

    def as_tool(self) -> StructuredTool:
        """Convert to a LangChain StructuredTool usable by OpenAI agents."""
        return StructuredTool.from_function(
            func=self.forward,
            name="retriever",
            description="Retrieves semantically similar documents from a FAISS vector index given a natural language query.",
        )

In [16]:
# Calculator Tool for comparisons
class CalculatorTool:
    """Tool to compute YoY or QoQ percentage changes from numeric data."""

    @staticmethod
    def forward(input_str: str) -> str:
        """
        Accepts a JSON string like:
        {
          "data": [{"fiscal_year": "2022", "opex": 7434.0}, ...],
          "metric_key": "opex",
          "period_key": "fiscal_year",
          "comparison_type": "yoy"
        }
        Returns: JSON string with computed results.
        """
        # Clean up formatting issues from the LLM
        input_str = input_str.strip().strip("`").strip("'").strip('"')

        try:
            params = json.loads(input_str)
        except Exception as e:
            return json.dumps({"error": f"Invalid input JSON: {str(e)}", "received": input_str})

        # Input Structure
        data = params.get("data", [])
        metric_key = params.get("metric_key", "opex")
        period_key = params.get("period_key", "fiscal_year")
        comparison_type = params.get("comparison_type", "yoy")

        # Handle dictionary-shaped data (like {"2022": 7434, ...})
        if isinstance(data, dict):
            data = [{period_key: k, metric_key: v} for k, v in data.items()]

        if not isinstance(data, list):
            return json.dumps({"error": "Expected 'data' to be a list or dict of year:value pairs."})

        # Compute percentage changes
        results = []
        for i, record in enumerate(data):
            if i == 0:
                pct_change = None
            else:
                prev_value = data[i - 1][metric_key]
                curr_value = record[metric_key]
                pct_change = ((curr_value - prev_value) / prev_value) * 100

            results.append({
                period_key: record[period_key],
                metric_key: record[metric_key],
                f"{comparison_type}_change (%)": round(pct_change, 2) if pct_change is not None else None,
                "units": record.get("units", "millions USD"),
            })

        return json.dumps(results, indent=2)

    def as_tool(self) -> StructuredTool:
        return StructuredTool.from_function(
            func=self.forward,
            name="calculator",
            description=(
                "Computes YoY or QoQ percentage changes. Input should be a JSON string containing data and comparison details."
            ),
        )

In [ ]:
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGSMITH_KEY')
# os.environ["LANGCHAIN_PROJECT"] = "AgentPerformance"

In [24]:
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small") # Can be 'text-embedding-3-large'

def build_faiss_index(chunked_docs):
  embedding_model = OpenAIEmbeddings(embedding_model) # Can be 'text-embedding-3-large'

  # FAISS Vector Index
  faiss_store = FAISS.from_documents(chunked_docs, embedding_model)
  faiss_store.save_local("faiss_index")

  # Load FAISS Vector Store
  faiss_store = FAISS.load_local(
      "faiss_index",
      embeddings=embedding_model,
      allow_dangerous_deserialization=True
  )
  return faiss_store

In [18]:
def create_agent(faiss_store):
  # Create Tools
  retriever_tool = RetrieverTool(faiss_store=faiss_store, top_k=12).as_tool()
  calculator_tool = CalculatorTool().as_tool()

  # Initialize OpenAI LLM
  openai_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

  # Create OpenAI-compatible agent that can use tools
  agent = initialize_agent(
      tools=[retriever_tool, calculator_tool],
      llm=openai_llm,
      agent_type=AgentType.OPENAI_FUNCTIONS,  # enables OpenAI’s function/tool calling
      verbose=True,
      handle_parsing_errors=True,
  )
  return agent

In [27]:
def create_system_message():
  return f"""
You are a financial analyst agent that can use tools.

Use the retriever tool when you need to fetch financial data. You can only call this tool ONCE.
Use the calculator tool when you need to compute YoY or QoQ comparisons.

Once done, return
1. The **final structured JSON output** in this format:
2. **Prose explanation**, converting the JSON output into a **formatted table**

{{
"query": "...",
"data_values": [...],
"computed_values": [...],
"citations": [{{"report": "...", "page": ..., "section": "..."}}],
"tools": ["<list the tools you actually used>"],
"tools_count": <total number of tools used>
}}

Guidelines:
- `data_values` contain the raw financial figures, corresponding fiscal years, and units retrieved directly from reports before any calculations.
- `computed_values` include the calculated results (e.g., YoY or QoQ changes) together with the corresponding values from data_values.
- Always include every period in `computed_values`, even if the change value is null.
"""

In [20]:
from langchain.callbacks.base import BaseCallbackHandler

class TimingCallback(BaseCallbackHandler):
    def __init__(self):
        self.tool_timings = []
        self.current_tool = None
        self.start_time = None
        self.llm_start = None
        self.reasoning_time = None

    def on_tool_start(self, serialized, input_str, **kwargs):
        self.current_tool = serialized.get("name", "UnknownTool")
        self.start_time = time.perf_counter()
        print(f"Tool start: {self.current_tool}")

    def on_tool_end(self, output, **kwargs):
        duration = time.perf_counter() - self.start_time
        print(f"Tool end: {self.current_tool} ({duration:.3f}s)")
        self.tool_timings.append((self.current_tool, duration))
        self.current_tool = None

    def on_llm_start(self, *args, **kwargs):
        self.llm_start = time.perf_counter()

    def on_llm_end(self, response, **kwargs):
        if self.llm_start:
            duration = time.perf_counter() - self.llm_start
            print(f"LLM reasoning/generation: {duration:.3f}s")
            self.reasoning_time = duration

In [ ]:
from langchain.schema import SystemMessage, HumanMessage


prof = Profiler(async_mode="disabled")
prof.start()
timing_callback = TimingCallback()

start_total = time.perf_counter()

# Build FAISS (if not yet built)
# faiss_store = build_faiss_index(chunked_docs)

# Load FAISS Vector Store
faiss_store = FAISS.load_local(
    "faiss_index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

agent = create_agent(faiss_store)

# Run the Agent with a Query
question = "Show Operating Expenses for the last 3 fiscal years, year-on-year comparison."
system_prompt = create_system_message()

response = agent.invoke({
    "input": [
        SystemMessage(content=system_prompt),
        HumanMessage(content=question)
    ]
}, config={"callbacks": [timing_callback]})
print(response)

elapsed_total = time.perf_counter() - start_total

prof.stop()
print(prof.output_text(unicode=True, color=True))
prof.last_session.save("02_Agent.pyisession")
html_output = prof.output_html()
with open("02_Agent.html", "w", encoding="utf-8") as f:
    f.write(html_output)
print(f"Total elapsed time: {elapsed_total}")




> Entering new AgentExecutor chain...
LLM reasoning/generation: 2.173s
To answer the question about Operating Expenses for the last three fiscal years with a year-on-year comparison, I need to retrieve the relevant financial data first. I will use the retriever tool to fetch the operating expenses data for the specified years.

Action: retriever  
Action Input: "Operating Expenses for the last 3 fiscal years year-on-year comparison"
Tool start: retriever
Tool end: retriever (0.432s)

Observation: ===== Document 1 =====
Table of Contents
Operating Expenses
Year Ended
January 29, January 30, $ %
2023 2022 Change Change
($ in millions)
Research and development expenses $ 7,339 $ 5,268 $ 2,071 39 %
% of revenue 27.2 % 19.6 %
Sales, general and administrative expenses 2,440 2,166 274 13 %
% of revenue 9.1 % 8.0 %
Acquisition termination cost 1,353 — 1,353 100 %
% of revenue 5.0 % — %
Total operating expenses $ 11,132 $ 7,434 $ 3,698 50 %
% of revenue 41.3 % 27.6 %
The increase in research

In [41]:
import json
import re

def extract_json_and_prose(response_text: str):
    """
    Extract both the JSON block and the prose explanation from an LLM response.
    Returns (parsed_json, prose_text).
    """
    pattern = re.compile(
        r"```json\s*(\{.*?\})\s*```(?:\s*\*\*Prose Explanation:\*\*\s*([\s\S]*))?",
        re.DOTALL
    )
    match = pattern.search(response_text)

    if not match:
        print("No JSON block found.")
        return None, None

    json_str = match.group(1).strip()
    prose = match.group(2).strip() if match.group(2) else ""

    try:
        data = json.loads(json_str)
    except json.JSONDecodeError as e:
        print(f"JSON parse error: {e}")
        data = None


    return data, prose

In [36]:
response['output']

'```json\n{\n  "query": "Operating Expenses for the last 3 fiscal years year-on-year comparison",\n  "data_values": [\n    {"fiscal_year": "2024", "opex": 11329, "units": "millions USD"},\n    {"fiscal_year": "2023", "opex": 11132, "units": "millions USD"},\n    {"fiscal_year": "2022", "opex": 7434, "units": "millions USD"}\n  ],\n  "computed_values": [\n    {"fiscal_year": "2024", "opex": 11329, "yoy_change (%)": null, "units": "millions USD"},\n    {"fiscal_year": "2023", "opex": 11132, "yoy_change (%)": -1.74, "units": "millions USD"},\n    {"fiscal_year": "2022", "opex": 7434, "yoy_change (%)": -33.22, "units": "millions USD"}\n  ],\n  "citations": [\n    {"report": "FY24_10K", "page": 40, "section": "Item 1A. Risk Factors"},\n    {"report": "FY23_10K", "page": 43, "section": "Item 1A. Risk"},\n    {"report": "FY22_10K", "page": 37, "section": "Item 1A. Risk Factors"}\n  ],\n  "tools": ["retriever", "calculator"],\n  "tools_count": 2\n}\n```\n\n**Prose Explanation:**\n\nThe table b

In [42]:
data, prose = extract_json_and_prose(response['output'])

In [51]:
data

{'query': 'Operating Expenses for the last 3 fiscal years year-on-year comparison',
 'data_values': [{'fiscal_year': '2024',
   'opex': 11329,
   'units': 'millions USD'},
  {'fiscal_year': '2023', 'opex': 11132, 'units': 'millions USD'},
  {'fiscal_year': '2022', 'opex': 7434, 'units': 'millions USD'}],
 'computed_values': [{'fiscal_year': '2024',
   'opex': 11329,
   'yoy_change (%)': None,
   'units': 'millions USD'},
  {'fiscal_year': '2023',
   'opex': 11132,
   'yoy_change (%)': -1.74,
   'units': 'millions USD'},
  {'fiscal_year': '2022',
   'opex': 7434,
   'yoy_change (%)': -33.22,
   'units': 'millions USD'}],
 'citations': [{'report': 'FY24_10K',
   'page': 40,
   'section': 'Item 1A. Risk Factors'},
  {'report': 'FY23_10K', 'page': 43, 'section': 'Item 1A. Risk'},
  {'report': 'FY22_10K', 'page': 37, 'section': 'Item 1A. Risk Factors'}],
 'tools': ['retriever', 'calculator'],
 'tools_count': 2}

In [49]:
prose = prose.encode('utf-8').decode('unicode_escape')
print(prose)

The table below summarizes the operating expenses for the last three fiscal years along with their year-on-year changes:

| Fiscal Year | Operating Expenses (in millions USD) | Year-on-Year Change (%) |
|-------------|--------------------------------------|-------------------------|
| 2024        | 11,329                               | N/A                     |
| 2023        | 11,132                               | -1.74                   |
| 2022        | 7,434                                | -33.22                  |

- In fiscal year 2024, the total operating expenses were $11,329 million, which represents a slight decrease of 1.74% compared to fiscal year 2023.
- Fiscal year 2023 saw a decrease of 33.22% in operating expenses compared to fiscal year 2022, where the expenses were $7,434 million. 

The data was retrieved from the financial reports for the respective years, and the year-on-year changes were calculated based on the operating expenses figures.


In [55]:
print("\n=== Summary ===")
for name, t in timing_callback.tool_timings:
    print(f"{name:<20} {t:.3f}s")
print(f"{'reasoning':<20} {timing_callback.reasoning_time:.3f}s")


=== Summary ===
retriever            0.432s
calculator           0.001s
reasoning            11.586s


## 5. Benchmark Runner

Run these 3 standardized queries. Produce JSON then prose answers with citations. These are the standardized queries.

*   Gross Margin Trend (or NIM if Bank)
    *   Query: "Report the Gross Margin (or Net Interest Margin, if a bank) over the last 5 quarters, with values."
    *   Expected Output: A quarterly table of Gross Margin % (or NIM % if bank).

*   Operating Expenses (Opex) YoY for 3 Years
    *   Query: "Show Operating Expenses for the last 3 fiscal years, year-on-year comparison."
    *   Expected Output: A 3-year Opex table (absolute numbers and % change).

*   Operating Efficiency Ratio
    *   Query: "Calculate the Operating Efficiency Ratio (Opex ÷ Operating Income) for the last 3 fiscal years, showing the working."
    *   Expected Output: Table with Opex, Operating Income, and calculated ratio for 3 years.


In [ ]:
# TODO: Implement benchmark runner


## 6. Instrumentation

Log timings: T_ingest, T_retrieve, T_rerank, T_reason, T_generate, T_total. Log tokens, cache hits, tools.

In [ ]:
# Example instrumentation schema
import pandas as pd
logs = pd.DataFrame(columns=['Query','T_ingest','T_retrieve','T_rerank','T_reason','T_generate','T_total','Tokens','CacheHits','Tools'])
logs

## 7. Optimizations

**Required Optimizations**

Each team must implement at least:
*   2 retrieval optimizations (e.g., hybrid BM25+vector, smaller embeddings, dynamic k).
*   1 caching optimization (query cache or ratio cache).
*   1 agentic optimization (plan pruning, parallel sub-queries).
*   1 system optimization (async I/O, batch embedding, memory-mapped vectors).

In [ ]:
# TODO: Implement optimizations

## 8. Results & Plots

Show baseline vs optimized. Include latency plots (p50/p95) and accuracy tables.

In [ ]:
# TODO: Generate plots with matplotlib
